## <a id = '0'> Índice </a>

* [**Entorno**](#1)  
   * [Librerías](#1d1)  
   * [Funciones](#1d2)  
   * [Constantes](#1d3)

* [**Lectura de datos**](#2)


## <a id = '1'> Entorno </a>
[índice](#0)

### <a id = '1d1'> Librerías </a>

In [1]:
import os
import pandas as pd
import numpy as np

import math
import itertools
# from config import data_folder

from scipy.stats import kurtosis, trim_mean


In [2]:
os.chdir("../")

### <a id = '1d2'> Funciones </a>

In [3]:
from src.utils import (lectura_carpetas_dict
                       , split_dataframe_into_chunks
                       , genera_df_acf_pacf_chunk
                       , get_dict_serie_summary_chunk
                       , patients_dict_ccf_chunk
                       , get_ccf_summary_chunk
                       )

### <a id = '1d3'> Constantes </a>

In [4]:
from config.cons import data_folder, list_signals

## <a id = '2'> Lectura de datos </a>
[índice](#0)

In [5]:
proyect_path = os.getcwd()
data_path = proyect_path + data_folder

In [6]:
# all_combinaciones = list(itertools.combinations(list_signals, 2))
all_combinaciones = list(itertools.combinations_with_replacement(list_signals, 2))


In [7]:
uni_combinacion = []
bi_combinacion = []
for col1, col2 in all_combinaciones:
    if col1 == col2:
        uni_combinacion.append(col1)
    else:
        bi_combinacion.append(col1 + "_" + col2)
dict_combinaciones = {
    "uni_combinacion": uni_combinacion,
    "bi_combinacion": bi_combinacion
}   

In [8]:
df_all = lectura_carpetas_dict(data_path)

In [9]:
df_mi = df_all["mi"]
df_sttc_mi = df_all["sttc_mi"]
df_sttc = df_all["sttc"]
df_other = df_all["other"]

In [10]:
dict_mi = {patient: split_dataframe_into_chunks(df_mi[patient], 100) for patient in df_mi.keys()}
df_mi_acf_pacf = genera_df_acf_pacf_chunk(dict_mi, list_signals, apply_diff=True)
df_mi_peak = get_dict_serie_summary_chunk(dict_mi, list_signals, period = 10, n_std = 1)
df_mi_ccf = patients_dict_ccf_chunk(dict_mi, all_combinaciones, max_lag=25)
mi_cff_features = get_ccf_summary_chunk(df_mi_ccf, dict_combinaciones, proportion_to_cut = 0.05)

In [15]:
df_mi_features = (
    df_mi_acf_pacf
    .merge(df_mi_peak, on=["patient_id", "label", "chunk"], how="inner")
    .merge(mi_cff_features, on=["patient_id", "label", "chunk"], how="inner")
)

cols = ['patient_id', 'label', 'chunk'] + [col for col in df_mi_features.columns if col not in ('patient_id', 'label', 'chunk')]

df_mi_features = df_mi_features[cols]

In [18]:
df_mi_features.to_csv("output/chunk_data/chunk_100/features/mi_features.csv", index=False)

In [19]:
del dict_mi, df_mi_acf_pacf, df_mi_peak, mi_cff_features

In [20]:
dict_sttc_mi = {patient: split_dataframe_into_chunks(df_sttc_mi[patient], 100) for patient in df_sttc_mi.keys()}
df_sttc_mi_acf_pacf = genera_df_acf_pacf_chunk(dict_sttc_mi, list_signals, apply_diff=True)
df_sttc_mi_peak = get_dict_serie_summary_chunk(dict_sttc_mi, list_signals, period = 10, n_std = 1)
df_sttc_mi_ccf = patients_dict_ccf_chunk(dict_sttc_mi, all_combinaciones, max_lag=25)
sttc_mi_cff_features = get_ccf_summary_chunk(df_sttc_mi_ccf, dict_combinaciones, proportion_to_cut = 0.05)

In [21]:
df_sttc_mi_features = (
    df_sttc_mi_acf_pacf
    .merge(df_sttc_mi_peak, on=["patient_id", "label", "chunk"], how="inner")
    .merge(sttc_mi_cff_features, on=["patient_id", "label", "chunk"], how="inner")
)

cols = ['patient_id', 'label', 'chunk'] + [col for col in df_sttc_mi_features.columns if col not in ('patient_id', 'label', 'chunk')]

df_sttc_mi_features = df_sttc_mi_features[cols]

In [22]:
df_sttc_mi_features.to_csv("output/chunk_data/chunk_100/features/sttc_mi_features.csv", index=False)


In [29]:
del dict_sttc_mi, df_sttc_mi_acf_pacf, df_sttc_mi_peak, df_sttc_mi_ccf

In [52]:
dict_sttc = {patient: split_dataframe_into_chunks(df_sttc[patient], 100) for patient in df_sttc.keys()}
df_sttc_acf_pacf = genera_df_acf_pacf_chunk(dict_sttc, list_signals, apply_diff=True)
df_sttc_peak = get_dict_serie_summary_chunk(dict_sttc, list_signals, period = 10, n_std = 1)
df_sttc_ccf = patients_dict_ccf_chunk(dict_sttc, all_combinaciones, max_lag=25)
sttc_cff_features = get_ccf_summary_chunk(df_sttc_ccf, dict_combinaciones, proportion_to_cut = 0.05)

In [53]:
df_sttc_features = (
    df_sttc_acf_pacf
    .merge(df_sttc_peak, on=["patient_id", "label", 'chunk'], how="inner")
    .merge(sttc_cff_features, on=["patient_id", "label", 'chunk'], how="inner")
)

cols = ['patient_id', 'label', 'chunk'] + [col for col in df_sttc_features.columns if col not in ('patient_id', 'label', 'chunk')]

df_sttc_features = df_sttc_features[cols]

In [54]:
df_sttc_features.to_csv("output/chunk_data/chunk_100/features/sttc_features.csv", index=False)

In [30]:
del dict_sttc, df_sttc_acf_pacf, df_sttc_peak, sttc_cff_features

In [58]:
dict_other = {patient: split_dataframe_into_chunks(df_other[patient], 100) for patient in df_other.keys()}
df_other_acf_pacf = genera_df_acf_pacf_chunk(dict_other, list_signals, apply_diff=True)
df_other_peak = get_dict_serie_summary_chunk(dict_other, list_signals, period = 10, n_std = 1)
df_other_ccf = patients_dict_ccf_chunk(dict_other, all_combinaciones, max_lag=25)
other_cff_features = get_ccf_summary_chunk(df_other_ccf, dict_combinaciones, proportion_to_cut = 0.05)

In [62]:
df_other_features = (
    df_other_acf_pacf
    .merge(df_other_peak, on=["patient_id", "label", 'chunk'], how="inner")
    .merge(other_cff_features, on=["patient_id", "label", 'chunk'], how="inner")
)

cols = ['patient_id', 'label', 'chunk'] + [col for col in df_other_features.columns if col not in ('patient_id', 'label', 'chunk')]

df_other_features = df_other_features[cols]

In [63]:
df_other_features.to_csv("output/chunk_data/chunk_100/features/other_features.csv", index=False)

In [31]:
del dict_other, df_other_acf_pacf, df_other_peak, other_cff_features

In [41]:
df_mi_features = pd.read_csv("output/chunk_data/chunk_100/features/mi_features.csv")
df_sttc_mi_features = pd.read_csv("output/chunk_data/chunk_100/features/sttc_mi_features.csv")
df_sttc_features = pd.read_csv("output/chunk_data/chunk_100/features/sttc_features.csv")
df_other_features = pd.read_csv("output/chunk_data/chunk_100/features/other_features.csv")

In [55]:
df_other_features.head()

,patient_id,label,chunk,acf_I_lag_1,acf_I_lag_2,acf_I_lag_3,acf_I_lag_4,acf_I_lag_5,pacf_I_lag_1,pacf_I_lag_2,...,n_cruces_cero_V5_V6,promedio_V5_V6,std_V5_V6,max_V5_V6,min_V5_V6,maxlag_V5_V6,minlag_V5_V6,kurtosis_V5_V6,trim_mean_V5_V6,norm_ccf
0,patient_213,other,chunk_1,0.370827,-0.355913,-0.34677,-0.137494,-0.020231,0.374611,-0.585791,...,6,0.157480,0.208917,0.879583,-0.069761,0,10,2.018088,0.138784,1.556835
1,patient_213,other,chunk_2,0.370827,-0.355913,-0.34677,-0.137494,-0.020231,0.374611,-0.585791,...,3,0.098584,0.205135,0.899276,-0.071251,0,19,4.831864,0.074282,0.666785
2,patient_213,other,chunk_3,0.370827,-0.355913,-0.34677,-0.137494,-0.020231,0.374611,-0.585791,...,8,0.024836,0.217150,0.864443,-0.188577,0,-25,5.808757,-0.000641,1.619320
3,patient_213,other,chunk_4,0.370827,-0.355913,-0.34677,-0.137494,-0.020231,0.374611,-0.585791,...,6,0.038183,0.222174,0.882733,-0.180917,0,25,5.293470,0.012821,1.548094
4,patient_213,other,chunk_5,0.370827,-0.355913,-0.34677,-0.137494,-0.020231,0.374611,-0.585791,...,4,0.021286,0.212465,0.881119,-0.195572,0,-21,6.512489,-0.004078,1.608042


In [32]:
patients = pd.read_csv("output/split_train_test/patients_90_05_05.csv").rename(columns={"patient": "patient_id", "class": "label"})

In [33]:
patients_train = patients[(patients["sample"] == "train")][["patient_id", "label"]]
patients_test = patients[(patients["sample"] == "test")][["patient_id", "label"]]
patients_val = patients[(patients["sample"] == "validation")][["patient_id", "label"]]

In [67]:
df_features = pd.concat([df_mi_features, df_sttc_mi_features, df_sttc_features, df_other_features], axis=0)

In [70]:
df_features.to_csv("output/chunk_data/chunk_100/features/all_features.csv", index=False)

In [36]:
del df_mi_features, df_sttc_mi_features, df_sttc_features, df_other_features

In [ ]:
df_features = pd.read_csv("output/chunk_data/chunk_100/features/all_features.csv")

In [74]:
features_train = df_features.merge(patients_train, on=["patient_id", "label"], how="inner")

In [76]:
features_train.to_csv("output/chunk_data/chunk_100/train_test_val/train_features.csv", index=False)

In [77]:
features_test = df_features.merge(patients_test, on=["patient_id", "label"], how="inner")

In [78]:
features_test.to_csv("output/chunk_data/chunk_100/train_test_val/test_features.csv", index=False)

In [79]:
features_val = df_features.merge(patients_val, on=["patient_id", "label"], how="inner")

In [80]:
features_val.to_csv("output/chunk_data/chunk_100/train_test_val/val_features.csv", index=False)

### Train test val 80 - 10 - 10

In [81]:
patients = pd.read_csv("output/split_train_test/patients_80_10_10.csv").rename(columns={"patient": "patient_id", "class": "label"})

In [82]:
patients_train = patients[(patients["sample"] == "train")][["patient_id", "label"]]
patients_test = patients[(patients["sample"] == "test")][["patient_id", "label"]]
patients_val = patients[(patients["sample"] == "validation")][["patient_id", "label"]]

In [83]:
df_features = pd.read_csv("output/chunk_data/chunk_100/features/all_features.csv")

In [84]:
features_train = df_features.merge(patients_train, on=["patient_id", "label"], how="inner")
features_train.to_csv("output/chunk_data/chunk_100/train_test_val/train_features.csv", index=False)

In [15]:
features_train.shape

(1919000, 834)

In [85]:
features_test = df_features.merge(patients_test, on=["patient_id", "label"], how="inner")
features_test.to_csv("output/chunk_data/chunk_100/train_test_val/test_features.csv", index=False)

In [86]:
features_val = df_features.merge(patients_val, on=["patient_id", "label"], how="inner")
features_val.to_csv("output/chunk_data/chunk_100/train_test_val/val_features.csv", index=False)